In [47]:
# !pip install --upgrade 
# !pip install --upgrade langchain-community openai python-dotenv pyyaml
# !pip install langchain-core==0.3.33 langchain==0.3.16 langchain-community==0.3.16 langchain-text-splitters==0.3.5 langchain-openai==0.3.3 pydantic==2.7.4

  Using cached langchain-0.3.16-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.7.4-py3-none-any.whl.metadata (109 kB)
Using cached langchain-0.3.16-py3-none-any.whl (1.0 MB)
   ---------------------------------------- 0.0/54.5 kB ? eta -:--:--
   ------- -------------------------------- 10.2/54.5 kB ? eta -:--:--
   ------------------------------------- -- 51.2/54.5 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 54.5/54.5 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/409.0 kB ? eta -:--:--
   --------------------------------------- 409.0/409.0 kB 12.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----------------------------- ---------- 1.4/1.9 MB 30.1 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 30.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.2
    Uninstalling pydantic_core-2.27.2:
      Succ

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.7.4 which is incompatible.
langchain-pinecone 0.1.0 requires langchain-core<0.2.0,>=0.1.40, but you have langchain-core 0.3.33 which is incompatible.


In [49]:
# !pip install anaconda-cloud-auth==0.7.2

  Attempting uninstall: anaconda-cloud-auth
    Found existing installation: anaconda-cloud-auth 0.1.4
    Uninstalling anaconda-cloud-auth-0.1.4:
      Successfully uninstalled anaconda-cloud-auth-0.1.4


In [57]:
import os
import datetime
from typing import Optional, Dict, List, Any, Tuple
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import yaml

In [60]:
load_dotenv()

class CodeTranslationSystem:
    def __init__(self, kb_path: Optional[str] = None):
        self.knowledge_base: Dict[str, Dict[str, List[str]]] = {}
        if kb_path:
            self.load_knowledge_base(kb_path)
        self.super_agent = self.create_super_agent()
        self.translation_agent = self.create_translation_agent()
        self.max_iterations = 3
        self.execution_log: List[Dict[str, Any]] = []
        
    def load_knowledge_base(self, file_path: str):
        """Load a knowledge base from a YAML file"""
        with open(file_path, 'r') as f:
            self.knowledge_base = yaml.safe_load(f)

    def _get_code_rules(self, target_lang: str) -> str:
        """Generate code rule description in target language"""
        if not self.knowledge_base:
            return ""
            
        rules = self.knowledge_base.get(target_lang, {})
        return "\n".join(
            [f"# {cat.upper()}\n" + "\n".join(f"- {item}" for item in items)
             for cat, items in rules.items() if cat != "analysis_rules"]
        )
    
    def _log_step(self, step_name: str, input_data: dict, output_data: Any):
        """Log each step's input and output"""
        self.execution_log.append({
            "step": step_name,
            "timestamp": datetime.datetime.now().isoformat(),
            "input": input_data,
            "output": output_data
        })
        
    def _get_analysis_rules(self, target_lang: str) -> str:
        """Get rules for requirements analysis"""
        if not self.knowledge_base:
            return ""
            
        analysis_rules = self.knowledge_base.get(target_lang, {}).get("analysis_rules", [])
        return "Special Rules:\n" + "\n".join([f"- {rule}" for rule in analysis_rules])

    def create_super_agent(self) -> LLMChain:
        """Create requirement analysis agent"""
        analysis_template = """You are a senior code analysis expert. Perform these tasks:
        1. Identify source programming language (C/C++/FORTRAN)
        2. Identify target language (CUDA/OpenMP/JAX)
        3. Extract code content needing conversion
        4. Analyze potential conversion challenges
        5. Generate code conversion task description
        
        {% if analysis_rules %}
        {{ analysis_rules }}
        {% endif %}

        User input: {{user_input}}

        Respond in this format:
        Source Language: [detected source language]
        Target Language: [detected target language]
        Code Content: [extracted code block]
        Potential Issues: 
        - [Issue1 description]
        - [Issue2 description]
        Task Description: "Convert the following [source] code to [target]:\n[code]"
        """
        prompt = PromptTemplate(
            template=analysis_template,
            input_variables=["user_input"],
            partial_variables={"analysis_rules": self._get_analysis_rules("generic")},
            template_format="jinja2"
        )
        return LLMChain(
            llm=ChatOpenAI(
                temperature=0,
                model="gpt-3.5-turbo"
            ),
            prompt=prompt,
            output_key="analysis"
        )

    def create_translation_agent(self) -> LLMChain:
        """Create code translation agent"""
        translation_template = """You are an HPC code conversion expert. Convert this {{source_lang}} code to {{target_lang}}:
        Requirements:
        1. Maintain identical algorithmic logic
        2. Follow target language's performance best practices
        3. Add necessary comments explaining modifications
        4. Ensure syntactic correctness

        {{code_input}}

        Return ONLY converted code without explanations.
        """
        prompt = PromptTemplate(
            template=translation_template,
            input_variables=["source_lang", "target_lang", "code_input"],
            template_format="jinja2"
        )
        return LLMChain(
            llm=ChatOpenAI(
                temperature=0,
                model="gpt-3.5-turbo"
            ),
            prompt=prompt,
            output_key="translated_code"
        )

    def parse_analysis(self, analysis: str) -> Tuple[str, str, str, List[str]]:
        """Enhanced analysis results interpretation"""
        source_lang = target_lang = code_content = ""
        potential_issues = []
        in_issues_section = False

        for line in analysis.split('\n'):
            line = line.strip()
            if line.startswith('Source Language:'):
                source_lang = line.partition(':')[2].strip()
            elif line.startswith('Target Language:'):
                target_lang = line.partition(':')[2].strip()
            elif line.startswith('Code Content:'):
                code_content = line.partition(':')[2].strip()
            elif line.lower().startswith('potential issues'):
                in_issues_section = True
            elif in_issues_section and line.startswith('-'):
                potential_issues.append(line[2:].strip())

        return source_lang, target_lang, code_content, potential_issues

    def validate_code(self, code: str, target_lang: str) -> str:
        """Code validation workflow"""
        code_rules = self._get_code_rules(target_lang)
        
        validation_template = """Review this {{target_lang}} code:
        {{code}}
        
        {% if code_rules %}
        Code Rules:
        {{code_rules}}
        {% endif %}

        Format your findings as:
        Issues Found: [Yes/No]
        Rule Violations:
        - [Rule1] violation description (line X)
        - [Rule2] violation description (line Y)
        Suggestions: 
        - [Suggestion1]
        - [Suggestion2]
        """
        prompt = PromptTemplate(
            template=validation_template,
            input_variables=["target_lang", "code"],
            partial_variables={"code_rules": code_rules},
            template_format="jinja2"
        )
        
        chain = LLMChain(
            llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
            prompt=prompt
        )
        result = chain.run(target_lang=target_lang, code=code)
        
        self._log_step(
            step_name="code_validation",
            input_data={"target_lang": target_lang, "code": code},
            output_data=result
        )
        return result

    def iterative_improvement(self, initial_code: str, target_lang: str) -> str:
        """Iterative code improvement"""
        improvement_template = """Improve the code based on this report:
        {{validation_result}}

        {% if code_rules %}
        Must follow these rules:
        {{code_rules}}
        {% endif %}

        Original Code:
        {{current_code}}

        Requirements:
        1. Strictly follow the suggestions
        2. Preserve functionality
        3. Return complete revised code
        """

        code_rules = self._get_code_rules(target_lang)
        current_code = initial_code
        iteration_log = []

        for iter_num in range(self.max_iterations):
            validation_result = self.validate_code(current_code, target_lang)

            iteration_data = {
                "iteration": iter_num + 1,
                "code_before": current_code,
                "validation_result": validation_result
            }

            if "Issues Found: No" in validation_result:
                iteration_data["action"] = "validation_passed"
                iteration_log.append(iteration_data)
                break

            # 修复关键点：使用PromptTemplate代替f-string
            prompt_template = PromptTemplate(
                template=improvement_template,
                input_variables=["validation_result", "current_code"],
                partial_variables={"code_rules": code_rules},
                template_format="jinja2"
            )

            improvement_prompt = prompt_template.format(
                validation_result=validation_result,
                current_code=current_code
            )

            self._log_step(
                step_name="improvement_request",
                input_data={
                    "validation_result": validation_result,
                    "previous_code": current_code
                },
                output_data=improvement_prompt
            )

            new_code = self.translation_agent.run({
                "source_lang": target_lang,
                "target_lang": target_lang,
                "code_input": improvement_prompt
            })

            self._log_step(
                step_name="improvement_response",
                input_data={"prompt": improvement_prompt},
                output_data=new_code
            )

            iteration_data.update({
                "action": "code_improved",
                "code_after": new_code
            })
            iteration_log.append(iteration_data)
            current_code = new_code

        self._log_step(
            step_name="iteration_summary",
            input_data={
                "initial_code": initial_code,
                "max_iterations": self.max_iterations,
                "target_language": target_lang
            },
            output_data={
                "final_code": current_code,
                "iterations": iteration_log,
                "total_iterations": len(iteration_log)
            }
        )

        return current_code

    def process_request(self, user_input: str) -> Dict:
        """Process user request with full logging"""
        self._log_step("user_input", {}, user_input)
        
        # Requirement Analysis
        analysis = self.super_agent.run(user_input=user_input)
        self._log_step("requirement_analysis", {"user_input": user_input}, analysis)
        
        # Parse Analysis
        source_lang, target_lang, code_content, potential_issues = self.parse_analysis(analysis)
        self._log_step(
            "analysis_parsing",
            {"raw_analysis": analysis},
            {
                "source_lang": source_lang,
                "target_lang": target_lang,
                "code_content": code_content,
                "potential_issues": potential_issues
            }
        )

        # Initial Translation
        translated_code = self.translation_agent.run({
            "source_lang": source_lang,
            "target_lang": target_lang,
            "code_input": code_content
        })
        self._log_step(
            "initial_translation",
            {
                "source": source_lang,
                "target": target_lang,
                "code": code_content
            },
            translated_code
        )

        # Iterative Improvement
        final_code = self.iterative_improvement(translated_code, target_lang)
        self._log_step(
            "final_result",
            {"iterations": self.max_iterations},
            final_code
        )

        return {
            "source_language": source_lang,
            "target_language": target_lang,
            "original_code": code_content,
            "translated_code": final_code,
            "execution_log": self.execution_log
        }

In [61]:
if __name__ == "__main__":
    system = CodeTranslationSystem("D:/Projects/HPCAgent/KB/code_rules.yaml")
    
    # Example Input
    user_input = """
    Please help me convert the following FORTRAN code into CUDA code:
    PROGRAM VECTOR_ADD
    INTEGER, PARAMETER :: N = 1000000
    REAL :: A(N), B(N), C(N)
    DO I = 1, N
        C(I) = A(I) + B(I)
    END DO
    END PROGRAM
    """
    
    result = system.process_request(user_input)
    print("Conversion Results:")
    print(result["translated_code"])
    
    print("\n=== Full Execution Log ===")
    for log in result["execution_log"]:
        print(f"\n[{log['step']}]")
        print("Input:", log.get("input"))
        print("Output:", log.get("output"))

Conversion Results:
```cuda
#include <stdio.h>

__global__ void vecAdd(float *a, float *b, float *c, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}

int main() {
    int n = 100000;
    float *h_a, *h_b, *h_c;
    float *d_a, *d_b, *d_c;
    
    cudaMallocManaged(&h_a, n * sizeof(float));
    cudaMallocManaged(&h_b, n * sizeof(float));
    cudaMallocManaged(&h_c, n * sizeof(float));
    
    for (int i = 0; i < n; i++) {
        h_a[i] = i;
        h_b[i] = i * 2;
    }
    
    int blockSize = 256;
    int gridSize = (n + blockSize - 1) / blockSize;
    
    vecAdd<<<gridSize, blockSize>>>(h_a, h_b, h_c, n);
    cudaDeviceSynchronize();
    
    for (int i = 0; i < 10; i++) {
        printf("%f + %f = %f\n", h_a[i], h_b[i], h_c[i]);
    }
    
    cudaFree(h_a);
    cudaFree(h_b);
    cudaFree(h_c);
    
    return 0;
}
```

=== Full Execution Log ===

[user_input]
Input: {}
Output: 
    Please help me convert the foll